# Get full matches info using match id

In [1]:
import time

from itertools import chain

import requests as req
import pandas as pd
import numpy as np

from settings import TOKEN

## Goal

Add the matches found to our database collection so that we can perform analysis based on it.

## 1. Check our matches id data

In [2]:
matches_id = pd.read_pickle("../raw_data/matches_id.pkl")
print(f"Has {matches_id.shape[0]} unique matches")
matches_id.tail()

Has 116503 unique matches


,gameId,timestamp
116498,4673330145,1592843085935
116499,4692632725,1593934544283
116500,4689849050,1593767761890
116501,4686798992,1593590521983
116502,4686448500,1593550099668


## 2. Make a request to the API so that we can check what kind of information we are getting as response
    
We should focus specially on the **participants** entries and clean it a bit, some columns are better for champion and matches statistics instead of player statistics, so lets split the data up.

In [3]:
match_id_try = matches_id["gameId"]

raw_matches = []

for id in match_id_try[:10]:
    res = req.get(f"https://euw1.api.riotgames.com/lol/match/v4/matches/{id}?api_key={TOKEN}")
    raw_matches.append(res.json())

In [4]:
raw_matches[0].keys()

dict_keys(['gameId', 'platformId', 'gameCreation', 'gameDuration', 'queueId', 'mapId', 'seasonId', 'gameVersion', 'gameMode', 'gameType', 'teams', 'participants', 'participantIdentities'])

## 3. Extract match information

We are looking for the following format 


| match_id |	region | date_created |	match_duration | patch | winner |
| --- | --- | --- | --- | --- | --- |
| 4671787510 | 	EUW1 |	2020-06-21 | 1557 |	10.12 | Red |

**Basic match info**

In [5]:
match_keys = ["gameId", "platformId", "gameCreation", "gameDuration", "gameVersion"]

In [369]:
matches_list = [{key: match[key] for key in match if key in match_keys} for match in raw_matches]
matches_list[0]

{'gameId': 4671787510,
 'platformId': 'EUW1',
 'gameCreation': 1592764299945,
 'gameDuration': 1557,
 'gameVersion': '10.12.325.6222'}

**Winners**

In [7]:
winners = [team["teamId"] for x in raw_matches for team in x["teams"] if team["win"] == "Win"]
winners

[200, 100, 200, 200, 100, 200, 200, 200, 200, 100]

**Merge the lists**

In [370]:
matches_df = pd.DataFrame(matches_list)
matches_df = matches_df.assign(winner=winners)
matches_df["winner"] = matches_df["winner"].replace({100:"Blue", 200: "Red"})
matches_df.head()

,gameId,platformId,gameCreation,gameDuration,gameVersion,winner
0,4671787510,EUW1,1592764299945,1557,10.12.325.6222,Red
1,4671790912,EUW1,1592761774006,1716,10.12.325.6222,Blue
2,4704868250,EUW1,1594599534398,1235,10.14.327.8505,Red
3,4718171384,EUW1,1595343579528,2227,10.14.327.8505,Red
4,4718095374,EUW1,1595341141852,1822,10.14.327.8505,Blue


**Clean up a bit the data to make it readable**

- Change gameCreation to DateTime format


In [371]:
matches_df["gameCreation"] = pd.to_datetime(matches_df["gameCreation"], unit="ms").dt.to_period("D")
matches_df.head()

,gameId,platformId,gameCreation,gameDuration,gameVersion,winner
0,4671787510,EUW1,2020-06-21,1557,10.12.325.6222,Red
1,4671790912,EUW1,2020-06-21,1716,10.12.325.6222,Blue
2,4704868250,EUW1,2020-07-13,1235,10.14.327.8505,Red
3,4718171384,EUW1,2020-07-21,2227,10.14.327.8505,Red
4,4718095374,EUW1,2020-07-21,1822,10.14.327.8505,Blue


**Change column names to a better format**

In [372]:
matches_df.rename(columns={
    "gameId": "match_id",
    "platformId": "region",
    "gameCreation": "date_created",
    "gameDuration": "match_duration",
    "gameVersion": "patch"
}, inplace=True)
matches_df.head()

,match_id,region,date_created,match_duration,patch,winner
0,4671787510,EUW1,2020-06-21,1557,10.12.325.6222,Red
1,4671790912,EUW1,2020-06-21,1716,10.12.325.6222,Blue
2,4704868250,EUW1,2020-07-13,1235,10.14.327.8505,Red
3,4718171384,EUW1,2020-07-21,2227,10.14.327.8505,Red
4,4718095374,EUW1,2020-07-21,1822,10.14.327.8505,Blue


**Change patch format to a double digit e.g. 10.12**

In [373]:
matches_df["patch"] = matches_df["patch"].str.slice(stop=5)
matches_df.head()

,match_id,region,date_created,match_duration,patch,winner
0,4671787510,EUW1,2020-06-21,1557,10.12,Red
1,4671790912,EUW1,2020-06-21,1716,10.12,Blue
2,4704868250,EUW1,2020-07-13,1235,10.14,Red
3,4718171384,EUW1,2020-07-21,2227,10.14,Red
4,4718095374,EUW1,2020-07-21,1822,10.14,Blue


## 4. Extract Champions data

**Get banned champions for each match**

| champion_id | match_id | banned | 
| --- | --- | --- |

In [12]:
print(raw_matches[0]["gameId"])
print([x["teamId"] for x in raw_matches[0]["teams"]])
teams = [x["teamId"] for x in raw_matches[0]["teams"]]
print("\nGet the bans for each team:\n")
print({n: raw_matches[0]["teams"][n]["bans"] for n in [i for i, x in enumerate(teams)]})

4671787510
[100, 200]

Get the bans for each team:

{0: [{'championId': 9, 'pickTurn': 1}, {'championId': 523, 'pickTurn': 2}, {'championId': 154, 'pickTurn': 3}, {'championId': 350, 'pickTurn': 4}, {'championId': 39, 'pickTurn': 5}], 1: [{'championId': 58, 'pickTurn': 6}, {'championId': 421, 'pickTurn': 7}, {'championId': 81, 'pickTurn': 8}, {'championId': 38, 'pickTurn': 9}, {'championId': 110, 'pickTurn': 10}]}


In [24]:
bans = []
for i, x in enumerate(raw_matches):
    for y in x["teams"]:
        for z in y["bans"]:
            bans.append({"champion_id": z["championId"],
                         "match_id": x["gameId"], 
                         "banned": 1})
bans[0]

{'champion_id': 9, 'match_id': 4671787510, 'banned': 1}

In [25]:
bans = pd.DataFrame(bans)
bans.head()

,champion_id,match_id,banned
0,9,4671787510,1
1,523,4671787510,1
2,154,4671787510,1
3,350,4671787510,1
4,39,4671787510,1


**Get picked champions for each match**

| champion_id | match_id | region | picked | lane | role | opponent | won | lost |
| --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 62 | 4671787510 | euw1 | 1 | JUNGLE | SOLO | 43 | 1 | 0 |

In [26]:
print(raw_matches[0]["gameId"])
print(raw_matches[0]["participants"][0]["championId"])
print(raw_matches[0]["participants"][0]["timeline"]["lane"])

4671787510
62
JUNGLE


**Get the data we want**

- ids, region, lane, opponents, win or lose, rune_set, item_set.

In [66]:
picks = []
for x in raw_matches:
    for i, y in enumerate(x["participants"]):
        picks.append({
            "champion_id": y["championId"],
            "match_id": x["gameId"],
            "region": x["platformId"],
            "picked": 1,
            "lane": y["timeline"]["lane"],
            "role": y["timeline"]["role"],
            "opponent": ([x["championId"] 
                              for x in x["participants"][5:]] 
                                 if i < 5 
                                 else [x["championId"] 
                                    for x in x["participants"][:5]]),
            "won": y["stats"]["win"],
            "lost": not y["stats"]["win"],
        })
picks[0]

{'champion_id': 62,
 'match_id': 4671787510,
 'region': 'EUW1',
 'picked': 1,
 'lane': 'JUNGLE',
 'role': 'NONE',
 'opponent': [55, 64, 51, 117, 164],
 'won': False,
 'lost': True}

In [67]:
picks = pd.DataFrame(picks)
picks.head()

,champion_id,match_id,region,picked,lane,role,opponent,won,lost
0,62,4671787510,EUW1,1,JUNGLE,NONE,"[55, 64, 51, 117, 164]",False,True
1,222,4671787510,EUW1,1,BOTTOM,DUO_CARRY,"[55, 64, 51, 117, 164]",False,True
2,412,4671787510,EUW1,1,BOTTOM,DUO_SUPPORT,"[55, 64, 51, 117, 164]",False,True
3,555,4671787510,EUW1,1,MIDDLE,SOLO,"[55, 64, 51, 117, 164]",False,True
4,54,4671787510,EUW1,1,TOP,SOLO,"[55, 64, 51, 117, 164]",False,True


**Replace True, False with 1, 0 respectively**

In [68]:
picks = picks.assign(won=np.where(picks["won"], 1, 0), lost=np.where(picks["lost"], 1, 0))

In [69]:
picks.head()

,champion_id,match_id,region,picked,lane,role,opponent,won,lost
0,62,4671787510,EUW1,1,JUNGLE,NONE,"[55, 64, 51, 117, 164]",0,1
1,222,4671787510,EUW1,1,BOTTOM,DUO_CARRY,"[55, 64, 51, 117, 164]",0,1
2,412,4671787510,EUW1,1,BOTTOM,DUO_SUPPORT,"[55, 64, 51, 117, 164]",0,1
3,555,4671787510,EUW1,1,MIDDLE,SOLO,"[55, 64, 51, 117, 164]",0,1
4,54,4671787510,EUW1,1,TOP,SOLO,"[55, 64, 51, 117, 164]",0,1


**Now we need to explode the dataframe to get each individual matchup**

In [70]:
picks = picks.explode(column="opponent").reset_index()
picks.drop(columns="index", inplace=True)
picks.head()

,champion_id,match_id,region,picked,lane,role,opponent,won,lost
0,62,4671787510,EUW1,1,JUNGLE,NONE,55,0,1
1,62,4671787510,EUW1,1,JUNGLE,NONE,64,0,1
2,62,4671787510,EUW1,1,JUNGLE,NONE,51,0,1
3,62,4671787510,EUW1,1,JUNGLE,NONE,117,0,1
4,62,4671787510,EUW1,1,JUNGLE,NONE,164,0,1


**Next we need to check the lane and role variables, we want this format for role**

 - TOP
 - JUNGLE
 - MIDDLE
 - BOTTOM
 - SUPPORT

In [71]:
support = picks.loc[picks["role"] == "DUO_SUPPORT"].index
middle = picks.loc[picks["role"] == "DUO"].index

picks.loc[picks.index[support], 'lane'] = "SUPPORT"
picks.loc[picks.index[middle], 'lane'] = "MIDDLE"

In [72]:
picks = picks.drop(columns="role")
picks.head()

,champion_id,match_id,region,picked,lane,opponent,won,lost
0,62,4671787510,EUW1,1,JUNGLE,55,0,1
1,62,4671787510,EUW1,1,JUNGLE,64,0,1
2,62,4671787510,EUW1,1,JUNGLE,51,0,1
3,62,4671787510,EUW1,1,JUNGLE,117,0,1
4,62,4671787510,EUW1,1,JUNGLE,164,0,1


In [75]:
[x for x in picks["lane"].unique()]

['JUNGLE', 'BOTTOM', 'SUPPORT', 'MIDDLE', 'TOP']

## 5. Extract players data

### 5.1 Player Info

- extract players information
- extract players champion information
- extract players lane information

In [189]:
players_info = pd.read_pickle("../raw_data/players_pool_account.pkl")
players_info = players_info.reindex(columns=["accountId", "summonerId", "summonerName", "tier", "rank", "leaguePoints", "wins", "losses"])
players_info = players_info.rename(columns={"accountId": "account_id",
                             "summonerId": "summoner_id",
                             "summonerName": "name",
                             "leaguePoints": "league_points"})
players_info.head()

,account_id,summoner_id,name,tier,rank,league_points,wins,losses
0,Xqb-V8M3R76oVcB07Xl9HIW1aQQtlFBdIc0SUN4QJMCvVpg,rvTHik0BLi0tKf0oBNH1LB-MrXNHgTYTIJp_xM9Jl22xfFpp,Meiran,DIAMOND,I,90,200,187
1,2O_DVP7Wz4HWs039wGEKgdNsSrt_E3mQv9SZnEEH-G2LYt4,Q1MxBiPaWxpvLUOcHTyUTGIRLoOsQ0pPW2GXJpFzlShKg4J_,Preciöus,DIAMOND,I,0,101,79
2,D3JuXBnzCjgcpOe3l-mfjOFv6_CypuLMKonCRtFWxxhYJl0,UHd2YrByzhM458uxwF_sR5saxARizi-quOa-S30OmFH41Or8,ˆˇˇˆˇˇˆ,DIAMOND,I,52,160,146
3,K1II1_OHR-OOdOvq_pqmDLqCJh7UH7A7RijWxD1erWu7KxY,JbkMW-xXj99fD7CesxP-4Q7HTbTVw_DjEAXhMuPMGqj1xEZt,Huskigodx,DIAMOND,I,3,264,261
4,6-ZpLloiZHOa7RjJSXDYC9N_835Hi6uIlzAe2fU5T7H4kIg,2RiE8ql63iwtSSYQYA_f84PtrCX_js2FrAWlOJfI4Wm83uES,Ghiosu,DIAMOND,I,30,448,434


**Getting the account info, champion used and role.**

In [256]:
info_keys = ["accountId", "summonerId", "currentPlatformId", "summonerName"]
print([x for x in raw_matches[0]["participantIdentities"][0]["player"].items() if x[0] in info_keys])
print(raw_matches[0]["participants"][0]["championId"])
print(raw_matches[0]["participants"][0]["timeline"]["lane"])
print(raw_matches[0]["participants"][0]["timeline"]["role"])

[('accountId', '7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKmPPWtbXHeOk'), ('summonerName', 'mental ƒortress'), ('summonerId', 'CGDTBnwxRTW_K9mtm5yvruEb4GHhdOKpNl09dBbBRy5-w-Uw'), ('currentPlatformId', 'EUW1')]
62
JUNGLE
NONE


In [417]:
participants = []
for x in raw_matches:
    for i, y in enumerate(x["participants"]):
        participants.append({
            "account_id": x["participantIdentities"][i]["player"]["accountId"],
            "summoner_id": x["participantIdentities"][i]["player"]["summonerId"],
            "region": x["participantIdentities"][i]["player"]["currentPlatformId"],
            "name": x["participantIdentities"][i]["player"]["summonerName"],
            "champion": y["championId"],
            "lane": y["timeline"]["lane"],
            "role": y["timeline"]["role"],
        })
participants[0]

{'account_id': '7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKmPPWtbXHeOk',
 'summoner_id': 'CGDTBnwxRTW_K9mtm5yvruEb4GHhdOKpNl09dBbBRy5-w-Uw',
 'region': 'EUW1',
 'name': 'mental ƒortress',
 'champion': 62,
 'lane': 'JUNGLE',
 'role': 'NONE'}

**Create a dataframe, and assign the right values to lane variable**

In [419]:
participants_df = pd.DataFrame(participants)

support = participants_df.loc[participants_df["role"] == "DUO_SUPPORT"].index
middle = participants_df.loc[participants_df["role"] == "DUO"].index
participants_df.loc[participants_df.index[support], 'lane'] = "SUPPORT"
participants_df.loc[participants_df.index[middle], 'lane'] = "MIDDLE"

participants_df = participants_df.drop(columns="role")
participants_df.head()

,account_id,summoner_id,region,name,champion,lane
0,7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKm...,CGDTBnwxRTW_K9mtm5yvruEb4GHhdOKpNl09dBbBRy5-w-Uw,EUW1,mental ƒortress,62,JUNGLE
1,fG8JDk5zVxKmoAMUqBIE8nbgMqzn8zuJrDJFPslxAg,C84ZjVnfQ-pwqcISLYNSop0JLFoR3smMwxdag22Budw,EUW1,NonoCASTER Hi,222,BOTTOM
2,Kc97-m0MqgpSk3DFoY17uq39_Roh9Qvi-xtoEFXPsMhEWPY,fW-WSP0bZEEIxieoBzCNP8EGmo9S-x7AR50mwNKBaVb32Iw,EUW1,eminemorslim,412,SUPPORT
3,WmwA8a6PWVm1SkA3JWpID3CDFAbqxjrsU9f345u3_qR12c...,7KhpW4cmmhIkojRE1Es6NnAZYxkqJsuaGFNbo_LR37Zh8OI-,EUW1,ˆKoichi,555,MIDDLE
4,4NI6_UJFRWXe6swXbvsN9dQl6ORPjOfU1EA7ybGJjuc,dkPzTIROBk6T6Ot7DD4hmd0YN0ogmfji_z-05s6mMZSM,EUW1,Minamas,54,TOP


**Now lets create frequency colums for each lane per account_id as a new table**

In [420]:
players_lane = participants_df[["account_id", "lane"]]
players_lane.head()

,account_id,lane
0,7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKm...,JUNGLE
1,fG8JDk5zVxKmoAMUqBIE8nbgMqzn8zuJrDJFPslxAg,BOTTOM
2,Kc97-m0MqgpSk3DFoY17uq39_Roh9Qvi-xtoEFXPsMhEWPY,SUPPORT
3,WmwA8a6PWVm1SkA3JWpID3CDFAbqxjrsU9f345u3_qR12c...,MIDDLE
4,4NI6_UJFRWXe6swXbvsN9dQl6ORPjOfU1EA7ybGJjuc,TOP


**Do the same for champions**

    Lets convert the champion ids to names, for this we need to request the champion list from the api!

In [348]:
champions = req.get("https://ddragon.leagueoflegends.com/cdn/10.14.1/data/en_US/champion.json").json()

In [421]:
champion_keys = champions["data"].keys()
champions_list = {}
for x in champions["data"]:
    champions_list[int(champions["data"][x]["key"])] = champions["data"][x]["name"]

champions_list
participants_df = participants_df.replace({"champion": champions_list})
participants_df["champion"].head()

0      Wukong
1        Jinx
2      Thresh
3        Pyke
4    Malphite
Name: champion, dtype: object

**Creating the DataFrame**

In [422]:
players_champions = participants_df[["account_id", "champion"]]
players_champions.head()

,account_id,champion
0,7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKm...,Wukong
1,fG8JDk5zVxKmoAMUqBIE8nbgMqzn8zuJrDJFPslxAg,Jinx
2,Kc97-m0MqgpSk3DFoY17uq39_Roh9Qvi-xtoEFXPsMhEWPY,Thresh
3,WmwA8a6PWVm1SkA3JWpID3CDFAbqxjrsU9f345u3_qR12c...,Pyke
4,4NI6_UJFRWXe6swXbvsN9dQl6ORPjOfU1EA7ybGJjuc,Malphite


**Isolate the account_info table**

    Drop the lane and champion columns and drop duplicated because we dont need them

In [423]:
participants_df = participants_df.drop(columns=["lane", "champion"])
participants_df.head(1)

,account_id,summoner_id,region,name
0,7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKm...,CGDTBnwxRTW_K9mtm5yvruEb4GHhdOKpNl09dBbBRy5-w-Uw,EUW1,mental ƒortress


In [424]:
participants_df = participants_df.drop_duplicates()
participants_df.head()

,account_id,summoner_id,region,name
0,7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKm...,CGDTBnwxRTW_K9mtm5yvruEb4GHhdOKpNl09dBbBRy5-w-Uw,EUW1,mental ƒortress
1,fG8JDk5zVxKmoAMUqBIE8nbgMqzn8zuJrDJFPslxAg,C84ZjVnfQ-pwqcISLYNSop0JLFoR3smMwxdag22Budw,EUW1,NonoCASTER Hi
2,Kc97-m0MqgpSk3DFoY17uq39_Roh9Qvi-xtoEFXPsMhEWPY,fW-WSP0bZEEIxieoBzCNP8EGmo9S-x7AR50mwNKBaVb32Iw,EUW1,eminemorslim
3,WmwA8a6PWVm1SkA3JWpID3CDFAbqxjrsU9f345u3_qR12c...,7KhpW4cmmhIkojRE1Es6NnAZYxkqJsuaGFNbo_LR37Zh8OI-,EUW1,ˆKoichi
4,4NI6_UJFRWXe6swXbvsN9dQl6ORPjOfU1EA7ybGJjuc,dkPzTIROBk6T6Ot7DD4hmd0YN0ogmfji_z-05s6mMZSM,EUW1,Minamas


**Update the players information table**

In [425]:
players_info = players_info.merge(participants_df, on=["account_id", "summoner_id", "name"], how="left")

players_info.head()

,account_id,summoner_id,name,tier,rank,league_points,wins,losses,region_x,region_y
0,Xqb-V8M3R76oVcB07Xl9HIW1aQQtlFBdIc0SUN4QJMCvVpg,rvTHik0BLi0tKf0oBNH1LB-MrXNHgTYTIJp_xM9Jl22xfFpp,Meiran,DIAMOND,I,90,200,187,NaN,NaN
1,2O_DVP7Wz4HWs039wGEKgdNsSrt_E3mQv9SZnEEH-G2LYt4,Q1MxBiPaWxpvLUOcHTyUTGIRLoOsQ0pPW2GXJpFzlShKg4J_,Preciöus,DIAMOND,I,0,101,79,EUW1,EUW1
2,D3JuXBnzCjgcpOe3l-mfjOFv6_CypuLMKonCRtFWxxhYJl0,UHd2YrByzhM458uxwF_sR5saxARizi-quOa-S30OmFH41Or8,ˆˇˇˆˇˇˆ,DIAMOND,I,52,160,146,EUW1,EUW1
3,K1II1_OHR-OOdOvq_pqmDLqCJh7UH7A7RijWxD1erWu7KxY,JbkMW-xXj99fD7CesxP-4Q7HTbTVw_DjEAXhMuPMGqj1xEZt,Huskigodx,DIAMOND,I,3,264,261,NaN,NaN
4,6-ZpLloiZHOa7RjJSXDYC9N_835Hi6uIlzAe2fU5T7H4kIg,2RiE8ql63iwtSSYQYA_f84PtrCX_js2FrAWlOJfI4Wm83uES,Ghiosu,DIAMOND,I,30,448,434,NaN,NaN


### 5.2 Laning stats @10

In [426]:
laning = []
for x in raw_matches:
    for i, y in enumerate(x["participants"]):
        laning.append({
            "account_id": x["participantIdentities"][i]["player"]["accountId"],
            "region": x["participantIdentities"][i]["player"]["currentPlatformId"],
            "champion": y["championId"],
            "lane": y["timeline"]["lane"],
            "role": y["timeline"]["role"],
            "xppm_10": y["timeline"]["xpPerMinDeltas"]["0-10"],
            "cspm_10": y["timeline"]["creepsPerMinDeltas"]["0-10"],
            "goldpm_10": y["timeline"]["goldPerMinDeltas"]["0-10"],
            "dmg_takenpm_10": y["timeline"]["damageTakenPerMinDeltas"]["0-10"],
        })
laning[0]

{'account_id': '7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKmPPWtbXHeOk',
 'region': 'EUW1',
 'champion': 62,
 'lane': 'JUNGLE',
 'role': 'NONE',
 'xppm_10': 267.3,
 'cspm_10': 0.5,
 'goldpm_10': 261.4,
 'dmg_takenpm_10': 654.3}

In [427]:
players_laning_stats = pd.DataFrame(laning)

support = players_laning_stats.loc[players_laning_stats["role"] == "DUO_SUPPORT"].index
middle = players_laning_stats.loc[players_laning_stats["role"] == "DUO"].index
players_laning_stats.loc[players_laning_stats.index[support], 'lane'] = "SUPPORT"
players_laning_stats.loc[players_laning_stats.index[middle], 'lane'] = "MIDDLE"

players_laning_stats = players_laning_stats.drop(columns="role")
players_laning_stats.head()

,account_id,region,champion,lane,xppm_10,cspm_10,goldpm_10,dmg_takenpm_10
0,7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKm...,EUW1,62,JUNGLE,267.3,0.5,261.4,654.3
1,fG8JDk5zVxKmoAMUqBIE8nbgMqzn8zuJrDJFPslxAg,EUW1,222,BOTTOM,342.6,6.9,345.3,243.3
2,Kc97-m0MqgpSk3DFoY17uq39_Roh9Qvi-xtoEFXPsMhEWPY,EUW1,412,SUPPORT,274.1,1.1,289.8,311.0
3,WmwA8a6PWVm1SkA3JWpID3CDFAbqxjrsU9f345u3_qR12c...,EUW1,555,MIDDLE,359.4,4.3,341.8,618.7
4,4NI6_UJFRWXe6swXbvsN9dQl6ORPjOfU1EA7ybGJjuc,EUW1,54,TOP,384.4,5.6,207.2,335.3


### 5.3 Combat stats


In [402]:
combat = []
for x in raw_matches:
    for i, y in enumerate(x["participants"]):
        combat.append({
            "account_id": x["participantIdentities"][i]["player"]["accountId"],
            "region": x["participantIdentities"][i]["player"]["currentPlatformId"],
            "champion": y["championId"],
            "lane": y["timeline"]["lane"],
            "role": y["timeline"]["role"],
            "dmg_total": y["stats"]["totalDamageDealtToChampions"],
            "healing_total": y["stats"]["totalHeal"],
            "units_healed": y["stats"]["totalUnitsHealed"],
            "damage_mitigated": y["stats"]["damageSelfMitigated"],
            "crowd_control": y["stats"]["totalTimeCrowdControlDealt"],
            "dmg_taken": y["stats"]["totalDamageTaken"],
            "first_blood": y["stats"]["firstBloodKill"],
            "first_blood_assist": y["stats"]["firstBloodAssist"],
        })
combat[0]

{'account_id': '7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKmPPWtbXHeOk',
 'region': 'EUW1',
 'champion': 62,
 'lane': 'JUNGLE',
 'role': 'NONE',
 'dmg_total': 8575,
 'healing_total': 6486,
 'units_healed': 1,
 'damage_mitigated': 14108,
 'crowd_control': 72,
 'dmg_taken': 22451,
 'first_blood': False,
 'first_blood_assist': False}

In [408]:
player_combat_stats = pd.DataFrame(combat)

support = player_combat_stats.loc[player_combat_stats["role"] == "DUO_SUPPORT"].index
middle = player_combat_stats.loc[player_combat_stats["role"] == "DUO"].index
player_combat_stats.loc[player_combat_stats.index[support], 'lane'] = "SUPPORT"
player_combat_stats.loc[player_combat_stats.index[middle], 'lane'] = "MIDDLE"

player_combat_stats = (player_combat_stats.assign(
                        first_blood=np.where(player_combat_stats["first_blood"], 1, 0),
                        first_blood_assist=np.where(player_combat_stats["first_blood_assist"], 1, 0)))
player_combat_stats = player_combat_stats.drop(columns="role")
player_combat_stats.head()

,account_id,region,champion,lane,dmg_total,healing_total,units_healed,damage_mitigated,crowd_control,dmg_taken,first_blood,first_blood_assist
0,7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKm...,EUW1,62,JUNGLE,8575,6486,1,14108,72,22451,0,0
1,fG8JDk5zVxKmoAMUqBIE8nbgMqzn8zuJrDJFPslxAg,EUW1,222,BOTTOM,15109,365,1,7401,92,16096,0,0
2,Kc97-m0MqgpSk3DFoY17uq39_Roh9Qvi-xtoEFXPsMhEWPY,EUW1,412,SUPPORT,7913,774,5,16784,125,16743,0,0
3,WmwA8a6PWVm1SkA3JWpID3CDFAbqxjrsU9f345u3_qR12c...,EUW1,555,MIDDLE,15786,4251,1,11752,152,21388,0,0
4,4NI6_UJFRWXe6swXbvsN9dQl6ORPjOfU1EA7ybGJjuc,EUW1,54,TOP,12793,979,1,34204,505,21185,0,0


### 5.4 Flair stats


In [430]:
flair = []
for x in raw_matches:
    for i, y in enumerate(x["participants"]):
        flair.append({
            "account_id": x["participantIdentities"][i]["player"]["accountId"],
            "region": x["participantIdentities"][i]["player"]["currentPlatformId"],
            "champion": y["championId"],
            "lane": y["timeline"]["lane"],
            "role": y["timeline"]["role"],
            "killing_sprees": y["stats"]["killingSprees"],
            "longest_time_alive": y["stats"]["longestTimeSpentLiving"],
            "double_kills": y["stats"]["doubleKills"],
            "triple_kills": y["stats"]["tripleKills"],
            "quadra_kills": y["stats"]["quadraKills"],
            "penta_kills": y["stats"]["pentaKills"],
        })
flair[0]

{'account_id': '7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKmPPWtbXHeOk',
 'region': 'EUW1',
 'champion': 62,
 'lane': 'JUNGLE',
 'role': 'NONE',
 'killing_sprees': 1,
 'longest_time_alive': 495,
 'double_kills': 0,
 'triple_kills': 0,
 'quadra_kills': 0,
 'penta_kills': 0}

In [432]:
player_flair_stats = pd.DataFrame(flair)

support = player_flair_stats.loc[player_flair_stats["role"] == "DUO_SUPPORT"].index
middle = player_flair_stats.loc[player_flair_stats["role"] == "DUO"].index
player_flair_stats.loc[player_flair_stats.index[support], 'lane'] = "SUPPORT"
player_flair_stats.loc[player_flair_stats.index[middle], 'lane'] = "MIDDLE"

player_flair_stats = player_flair_stats.drop(columns="role")
player_flair_stats.head()

,account_id,region,champion,lane,killing_sprees,longest_time_alive,double_kills,triple_kills,quadra_kills,penta_kills
0,7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKm...,EUW1,62,JUNGLE,1,495,0,0,0,0
1,fG8JDk5zVxKmoAMUqBIE8nbgMqzn8zuJrDJFPslxAg,EUW1,222,BOTTOM,2,475,0,0,0,0
2,Kc97-m0MqgpSk3DFoY17uq39_Roh9Qvi-xtoEFXPsMhEWPY,EUW1,412,SUPPORT,1,807,0,0,0,0
3,WmwA8a6PWVm1SkA3JWpID3CDFAbqxjrsU9f345u3_qR12c...,EUW1,555,MIDDLE,2,358,0,0,0,0
4,4NI6_UJFRWXe6swXbvsN9dQl6ORPjOfU1EA7ybGJjuc,EUW1,54,TOP,1,726,0,0,0,0


### 5.5 Objective stats

In [434]:
objectives = []
for x in raw_matches:
    for i, y in enumerate(x["participants"]):
        objectives.append({
            "account_id": x["participantIdentities"][i]["player"]["accountId"],
            "region": x["participantIdentities"][i]["player"]["currentPlatformId"],
            "champion": y["championId"],
            "lane": y["timeline"]["lane"],
            "role": y["timeline"]["role"],
            "dmg_to_objectives": y["stats"]["damageDealtToObjectives"],
            "dmg_to_turrets": y["stats"]["damageDealtToTurrets"],
            "total_cs": y["stats"]["totalMinionsKilled"],
            "jungle_cs": y["stats"]["neutralMinionsKilled"],
            "jungle_invaded": y["stats"]["neutralMinionsKilledEnemyJungle"],
            "wards_placed": y["stats"]["wardsPlaced"],
            "wards_killed": y["stats"]["wardsKilled"],
        })
objectives[0]

{'account_id': '7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKmPPWtbXHeOk',
 'region': 'EUW1',
 'champion': 62,
 'lane': 'JUNGLE',
 'role': 'NONE',
 'dmg_to_objectives': 4718,
 'dmg_to_turrets': 119,
 'total_cs': 32,
 'jungle_cs': 57,
 'jungle_invaded': 4,
 'wards_placed': 6,
 'wards_killed': 6}

In [435]:
player_objective_stats = pd.DataFrame(objectives)

support = player_objective_stats.loc[player_objective_stats["role"] == "DUO_SUPPORT"].index
middle = player_objective_stats.loc[player_objective_stats["role"] == "DUO"].index
player_objective_stats.loc[player_objective_stats.index[support], 'lane'] = "SUPPORT"
player_objective_stats.loc[player_objective_stats.index[middle], 'lane'] = "MIDDLE"

player_objective_stats = player_objective_stats.drop(columns="role")
player_objective_stats.head()

,account_id,region,champion,lane,dmg_to_objectives,dmg_to_turrets,total_cs,jungle_cs,jungle_invaded,wards_placed,wards_killed
0,7UTpYZvoj06Si113SIlBe-jyteHrh-XRaYzuYfXWentoKm...,EUW1,62,JUNGLE,4718,119,32,57,4,6,6
1,fG8JDk5zVxKmoAMUqBIE8nbgMqzn8zuJrDJFPslxAg,EUW1,222,BOTTOM,4964,4964,190,13,0,6,2
2,Kc97-m0MqgpSk3DFoY17uq39_Roh9Qvi-xtoEFXPsMhEWPY,EUW1,412,SUPPORT,342,342,23,0,0,31,2
3,WmwA8a6PWVm1SkA3JWpID3CDFAbqxjrsU9f345u3_qR12c...,EUW1,555,MIDDLE,1646,1264,88,0,0,4,3
4,4NI6_UJFRWXe6swXbvsN9dQl6ORPjOfU1EA7ybGJjuc,EUW1,54,TOP,1143,1143,150,0,0,9,2


# On the next chapter I clean all of this mess into functions